In [ ]:
# 기사 목록
# euc-kr 코딩

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 중복 확인을 위한 집합 선언
collected_urls = set()

def fetch_search_results(keyword, start_date, end_date):
    base_url = "http://www.kwangju.co.kr/searchs.php?mode=Y&searchword="
    results = []
    page = 1

    while True:
        print(f"{keyword} {page} 페이지 수집 중...")
        encoded_keyword = urllib.parse.quote(keyword.encode('euc-kr'))
        search_url = base_url + encoded_keyword + f"&s_category=TC&section=&s_day={start_date}&e_day={end_date}&page={page}"
        driver.get(search_url)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        article_list = soup.find_all('td', style="padding:10px 0 10px 0")
        if not article_list:
            break

        new_articles_found = False

        for td in article_list:
            section_font = td.find('font', color='#CC0033')
            section = section_font.text.strip('[]') if section_font else ''

            a_tag = td.find('a', class_='sublist')
            if a_tag:
                url = "http://www.kwangju.co.kr" + a_tag['href'].split("&search=")[0]
                title = a_tag.get_text(strip=True)
                date = td.find('span', class_='date').text.replace(' ', '').strip() if td.find('span', class_='date') else ''

                # 중복 체크 및 새로운 기사 발견 여부
                if url not in collected_urls:
                    collected_urls.add(url)
                    new_articles_found = True

                    results.append({
                        'media': '광주일보',
                        'title': title,
                        'date': date,
                        'section': section,
                        'article_type': '',
                        'writer': '',
                        'page': '',
                        'url': url,
                        'content': ''
                    })

        if not new_articles_found:
            break

        page += 1

    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

# 키워드와 검색 기간 설정
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

all_results = []

try:
    for keyword in keywords:
        all_results.extend(fetch_search_results(keyword, start_date, end_date))

    # 수집된 기사 수 출력
    print(f"Total number of articles: {len(all_results)}")

    save_to_excel(all_results, f'2광주일보 {start_date}~{end_date}.xlsx')

finally:
    driver.quit()



In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 본문 추출
        article_body = soup.find('div', class_="read")

        # 서브 타이틀 추출 (div.rtitle2)
        subtitle = ""
        subtitle_tag = soup.find('div', class_='rtitle2')
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 서브 타이틀을 본문에서 제거

        content = ""
        if article_body:
            # 'caption' 클래스를 가진 모든 태그 제거
            for caption in article_body.find_all('table', align='center'):
                caption.decompose()
            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""

        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

try:
    # 각 행의 URL에서 본문 내용을 스크랩하여 content 및 서브 타이틀을 저장
    for index, row in df.iterrows():
        print(f"Processing {index + 1}/{total_articles}: {row['url']}")
        content, subtitle = get_content_and_subtitle_from_url(row['url'])

        if content is None:
            error_log.append(row['url'])

        df.at[index, 'content'] = content
        df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가

        # 100개마다 임시 파일 저장
        if (index + 1) % 100 == 0:
            df.to_excel(temp_output_path, index=False)
            print(f"임시 파일을 저장했습니다: {temp_output_path}")

finally:
    # 웹드라이버 종료
    driver.quit()

# 최종 파일 저장
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)
print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
